In [1]:
from __future__ import print_function


from tqdm import tqdm
import os
import sys
import time
import random
import pprint
import datetime
import dateutil.tz
import argparse
import numpy as np
import pandas as pd
from misc.config import Config
cfg = Config()
import pickle
import torch
import torchvision.transforms as transforms
pd.options.display.max_colwidth=0

In [26]:
proj_ds=pd.read_csv('../data/indiana_projections.csv')
repo_ds=pd.read_csv('../data/indiana_reports.csv')

# display(proj_ds.head(5),proj_ds.shape)
# display(repo_ds.sort_values(by='uid').head(5),repo_ds.shape)

merged_ds = pd.merge(left=repo_ds,right=proj_ds, how='inner', on='uid')
# display(merged_ds.head())
frontal_ds = merged_ds[merged_ds['projection']=='Frontal']
frontal_ds

,uid,MeSH,Problems,image,indication,comparison,findings,impression,filename,projection
0,1,normal,normal,Xray Chest PA and Lateral,Positive TB test,None.,The cardiac silhouette and mediastinum size ar...,Normal chest x-XXXX.,1_IM-0001-4001.dcm.png,Frontal
2,2,Cardiomegaly/borderline;Pulmonary Artery/enlarged,Cardiomegaly;Pulmonary Artery,"Chest, 2 views, frontal and lateral",Preop bariatric surgery.,None.,Borderline cardiomegaly. Midline sternotomy XX...,No acute pulmonary findings.,2_IM-0652-1001.dcm.png,Frontal
4,3,normal,normal,Xray Chest PA and Lateral,"rib pain after a XXXX, XXXX XXXX steps this XX...",NaN,NaN,"No displaced rib fractures, pneumothorax, or p...",3_IM-1384-1001.dcm.png,Frontal
6,4,"Pulmonary Disease, Chronic Obstructive;Bullous...","Pulmonary Disease, Chronic Obstructive;Bullous...","PA and lateral views of the chest XXXX, XXXX a...",XXXX-year-old XXXX with XXXX.,None available,There are diffuse bilateral interstitial and a...,1. Bullous emphysema and interstitial fibrosis...,4_IM-2050-1001.dcm.png,Frontal
8,5,Osteophyte/thoracic vertebrae/multiple/small;T...,Osteophyte;Thickening;Lung,Xray Chest PA and Lateral,Chest and nasal congestion.,NaN,The cardiomediastinal silhouette and pulmonary...,No acute cardiopulmonary abnormality.,5_IM-2117-1003002.dcm.png,Frontal
...,...,...,...,...,...,...,...,...,...,...
7456,3995,Lung/hyperdistention/mild;Diaphragm/bilateral/...,Lung;Diaphragm;Cicatrix;Pulmonary Atelectasis;...,Xray Chest PA and Lateral,"Nausea, vomiting x2 weeks. Dialysis patient.","XXXX, XXXX.",The cardiomediastinal silhouette and pulmonary...,1. Interval resolution of bibasilar airspace d...,3995_IM-2046-1001.dcm.png,Frontal
7458,3996,Spine/degenerative,Spine,Xray Chest PA and Lateral,NaN,None.,The lungs are clear. Heart size is normal. No ...,Clear lungs. No acute cardiopulmonary abnormal...,3996_IM-2047-1001.dcm.png,Frontal
7460,3997,Opacity/lung/upper lobe/right/round/small;Gran...,Opacity;Granuloma,PA and lateral views of the chest.,XXXX-year-old male with positive PPD.,None available.,"Heart size within normal limits. Small, nodula...","No acute findings, no evidence for active TB.",3997_IM-2048-1001.dcm.png,Frontal
7462,3998,normal,normal,"PA and lateral chest XXXX, XXXX XXXX comparis...",tuberculosis positive PPD,NaN,NaN,Heart size is normal and the lungs are clear.,3998_IM-2048-1001.dcm.png,Frontal


In [409]:
with open('../data/cleaned_dataset.pickle','rb') as f:
    dataset = pickle.load(f)

In [411]:
dataset['data_split'].keys()

dict_keys(['train_uids', 'val_uids', 'test_uids', 'val_test_uids'])

In [382]:
train_uids = dataset['data_split']['train_uids']
val_uids = dataset['data_split']['val_uids']
test_uids = dataset['data_split']['test_uids']
len(train_uids),len(val_uids),len(test_uids)

(3166, 200, 300)

In [383]:
train_uids_cls = list(set(dataset['classes'].keys()).intersection(set(train_uids)))
np.array(train_uids_cls)

array([   1,    2,    3, ..., 3997, 3998, 3999])

In [384]:
val_uids_cls = list(set(dataset['classes'].keys()).intersection(set(val_uids)))

In [385]:
test_uids_cls = list(set(dataset['classes'].keys()).intersection(set(test_uids)))

In [407]:
val_test_uids = np.array(list(val_uids)+list(test_uids))
dataset['data_split']['val_test_uids'] = val_test_uids

In [402]:

uid_prob_mapping = {}
# uid_mesh_mapping = {}
for uid in tqdm(val_test_uids):
    problems = frontal_ds[frontal_ds.uid==uid]['Problems'].values[0].lower().strip().split(';')
    p_probs = []
    for cls in iu_classes:
        for p in problems:
            if cls in p:
                p_probs+=[cls]
    if (len(p_probs)==0):
        p_probs = ['no finding']
#     p_probs = [x for x in problems if x in iu_classes]
#     meshs = repo_ds[repo_ds.uid==key]['MeSH'].values[0].lower().split(';')
    uid_prob_mapping[uid]=p_probs
#     uid_mesh_mapping[key]=meshs

100%|██████████| 500/500 [00:00<00:00, 1859.97it/s]


In [403]:
iu_problems = []
# iu_meshs = []
for key in tqdm(uid_prob_mapping.keys()):
    iu_problems+=uid_prob_mapping[key]
#     iu_meshs+=uid_mesh_mapping[key]

100%|██████████| 500/500 [00:00<00:00, 1083239.67it/s]


In [413]:
{'no finding':0
 ,'atelectasis':1
 ,'cardiomegaly':2
 ,'effusion':3
 ,'emphysema':4
 ,'infiltrate':5
 ,'nodule':6
 ,'thickening':7}

{'no finding': 0,
 'atelectasis': 1,
 'cardiomegaly': 2,
 'effusion': 3,
 'emphysema': 4,
 'infiltrate': 5,
 'nodule': 6,
 'thickening': 7}

In [404]:
iu_prob_count = {}
for i in range(len(iu_problems)):    
    if iu_problems[i] not in iu_prob_count.keys():
        iu_prob_count[iu_problems[i]] = 1
    else:
        iu_prob_count[iu_problems[i]] += 1

In [405]:
df_iu_prob_count = pd.DataFrame.from_dict(iu_prob_count,orient='index',columns=['count']).reset_index().sort_values(by=['index','count'], ascending=True)
'test',df_iu_prob_count,df_iu_prob_count.shape

('test',
           index  count
 1   atelectasis     45
 4  cardiomegaly     29
 3      effusion     17
 5     emphysema     18
 6    infiltrate      8
 0    no finding    406
 2        nodule      9
 7    thickening      7,
 (8, 2))

In [394]:
df_iu_prob_count = pd.DataFrame.from_dict(iu_prob_count,orient='index',columns=['count']).reset_index().sort_values(by=['index','count'], ascending=True)
'val',df_iu_prob_count,df_iu_prob_count.shape

('val',
           index  count
 2   atelectasis      9
 1  cardiomegaly     14
 4      effusion      8
 3     emphysema      7
 6    infiltrate      2
 0    no finding    165
 5        nodule      4
 7    thickening      2,
 (8, 2))

In [399]:
df_iu_prob_count = pd.DataFrame.from_dict(iu_prob_count,orient='index',columns=['count']).reset_index().sort_values(by=['index','count'], ascending=True)
'train',df_iu_prob_count,df_iu_prob_count.shape

('train',
           index  count
 4   atelectasis    265
 1  cardiomegaly    290
 5      effusion    137
 2     emphysema     86
 7    infiltrate     53
 0    no finding   2422
 6        nodule    103
 3    thickening     46,
 (8, 2))

In [334]:
iu_classes = [
    'atelectasis'
    ,'cardiomegaly'
    ,'effusion'
    ,'infiltrate'
    ,'nodule'
    ,'emphysema'
    ,'thickening'
]
len(iu_classes)

7

In [374]:
uid_prob_mapping = {}
# uid_mesh_mapping = {}
for uid in tqdm(dataset['data_dict'].keys()):
    problems = frontal_ds[frontal_ds.uid==uid]['Problems'].values[0].lower().strip().split(';')
    p_probs = []
    for cls in iu_classes:
        for p in problems:
            if cls in p:
                p_probs+=[cls]
    if (len(p_probs)==0):
        p_probs = ['no finding']
#     p_probs = [x for x in problems if x in iu_classes]
#     meshs = repo_ds[repo_ds.uid==key]['MeSH'].values[0].lower().split(';')
    uid_prob_mapping[uid]=[p_probs]
#     uid_mesh_mapping[key]=meshs

100%|██████████| 3666/3666 [00:01<00:00, 2069.58it/s]


In [375]:
df_iu_classes = pd.DataFrame.from_dict(uid_prob_mapping,orient='index',columns=['classes'])
df_iu_classes

,classes
1,[no finding]
2,[cardiomegaly]
3,[no finding]
4,[emphysema]
5,[thickening]
...,...
3995,"[atelectasis, emphysema]"
3996,[no finding]
3997,[no finding]
3998,[no finding]


In [376]:
dataset['classes'] = df_iu_classes.to_dict()['classes']
dataset['classes']

{1: ['no finding'],
 2: ['cardiomegaly'],
 3: ['no finding'],
 4: ['emphysema'],
 5: ['thickening'],
 6: ['no finding'],
 7: ['atelectasis'],
 8: ['no finding'],
 9: ['no finding'],
 10: ['no finding'],
 11: ['no finding'],
 12: ['no finding'],
 13: ['no finding'],
 14: ['no finding'],
 15: ['no finding'],
 17: ['no finding'],
 18: ['effusion'],
 19: ['no finding'],
 20: ['no finding'],
 21: ['no finding'],
 22: ['no finding'],
 23: ['no finding'],
 24: ['no finding'],
 25: ['effusion', 'effusion', 'emphysema'],
 26: ['no finding'],
 27: ['no finding'],
 28: ['atelectasis'],
 29: ['cardiomegaly'],
 30: ['no finding'],
 31: ['no finding'],
 32: ['no finding'],
 33: ['no finding'],
 34: ['no finding'],
 35: ['emphysema'],
 36: ['no finding'],
 37: ['no finding'],
 38: ['no finding'],
 39: ['no finding'],
 40: ['emphysema'],
 41: ['no finding'],
 42: ['no finding'],
 43: ['no finding'],
 44: ['no finding'],
 45: ['cardiomegaly'],
 46: ['no finding'],
 47: ['thickening'],
 48: ['nodule'],


In [338]:
# mimic_diseases = ['Atelectasis'
# ,'Cardiomegaly'
# ,'Consolidation'
# ,'Edema'
# ,'Enlarged Cardiomediastinum'
# ,'Fracture'
# ,'Lung Lesion'
# ,'Lung Opacity'
# ,'No Finding'
# ,'Pleural Effusion'
# ,'Pleural Other'
# ,'Pneumonia'
# ,'Pneumothorax'
# ,'Support Devices']

In [339]:
# len(mimic_diseases)

In [340]:
iu_problems = []
# iu_meshs = []
for key in tqdm(uid_prob_mapping.keys()):
    iu_problems+=uid_prob_mapping[key]
#     iu_meshs+=uid_mesh_mapping[key]

100%|██████████| 3666/3666 [00:00<00:00, 1394424.45it/s]


In [341]:
len(set(iu_problems))

8

In [342]:
iu_prob_count = {}
for i in range(len(iu_problems)):
    
    if iu_problems[i] not in iu_prob_count.keys():
        iu_prob_count[iu_problems[i]] = 1
    else:
        iu_prob_count[iu_problems[i]] += 1

In [343]:
df_iu_prob_count = pd.DataFrame.from_dict(iu_prob_count,orient='index',columns=['count']).reset_index().sort_values(by=['index','count'], ascending=True)

In [345]:
df_iu_prob_count

,index,count
4,atelectasis,310
1,cardiomegaly,319
5,effusion,154
2,emphysema,104
7,infiltrate,61
0,no finding,2828
6,nodule,112
3,thickening,53


In [119]:
pd.options.display.max_rows=2000